In [8]:
import nltk
import numpy as np
import pandas as pd
import re
import string
import matplotlib.pyplot as plt 
nltk.download('twitter_samples')
nltk.download('stopwords')
from nltk.corpus import twitter_samples
from nltk.corpus import stopwords      
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

[nltk_data] Downloading package twitter_samples to /home/neelraj-
[nltk_data]     reddy/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to /home/neelraj-
[nltk_data]     reddy/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Preparing the dataset

In [12]:
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

tweets = all_positive_tweets + all_negative_tweets
len(tweets)

10000

In [14]:
# make a numpy array representing labels of the tweets
labels = np.append(np.ones((len(all_positive_tweets))), np.zeros((len(all_negative_tweets))))
labels

array([1., 1., 1., ..., 0., 0., 0.])

In [15]:
# split the data into two pieces, one for training and one for testing (validation set) 

test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg 
test_x = test_pos + test_neg

In [16]:
# combine positive and negative labels

train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

In [17]:
# Print the shape train and test sets

print("train_y.shape = " + str(train_y.shape))
print("test_y.shape = " + str(test_y.shape))

train_y.shape = (8000, 1)
test_y.shape = (2000, 1)


# Preprocessing the data

1. Remove stop words
2. Remove URL's and tag's
3. remove punctuations
4. Then others like retweets using RT, #'s, stock market tickers like $ 

In [19]:
def preprocess_tweet(tweet):

    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')

    tweet = re.sub('\$\w*',"",tweet)      #-----> removes any word starting with dollar sign

    tweet = re.sub(r'^RT[\s]+', '', tweet)     #-----> removes RT   and following whitespace

    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)   #-----> remove hyperlink

    tweet = re.sub(r'#', '', tweet)   #-----> remove # from the text

    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):  # remove punctuation
            # tweets_clean.append(word)
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)

    return tweets_clean

In [22]:
print(train_neg[510])
print(preprocess_tweet(train_neg[510]))

@borapls it was white washed dude to the Polaroid effect :-(
['white', 'wash', 'dude', 'polaroid', 'effect', ':-(']


In [23]:
def build_freqs(tweets, ys):
    
    # tweets: a list of tweets
    # ys: an m x 1 array with the sentiment label of each tweet (either 0 or 1)
   
    yslist = np.squeeze(ys).tolist()

    freqs = {}
    for y, tweet in zip(yslist, tweets):
        for word in preprocess_tweet(tweet):
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1
    
    # freqs: a dictionary mapping each (word, sentiment) pair to its frequency
    
    return freqs 

In [24]:
# create frequency dictionary
freqs = build_freqs(tweets, labels)

# check data type
print(f'type(freqs) = {type(freqs)}')

# check length of the dictionary
print(f'len(freqs) = {len(freqs)}')

type(freqs) = <class 'dict'>
len(freqs) = 13065


In [29]:
keys=[]
for key in freqs.keys():
    keys.append(key[0])

keys=list(set(keys))

In [31]:
# each element consist of a sublist with this pattern: [<word>, <positive_count>, <negative_count>]
data = []

# loop through our selected words
for word in keys:
    
    # initialize positive and negative counts
    pos = 0
    neg = 0
    
    # retrieve number of positive counts
    if (word, 1) in freqs:
        pos = freqs[(word, 1)]
        
    # retrieve number of negative counts
    if (word, 0) in freqs:
        neg = freqs[(word, 0)]
        
    # append the word counts to the table
    data.append([word, pos, neg])
    
data

[['squad', 1, 0],
 ['buy', 11, 17],
 ['back', 98, 122],
 ['jumma', 5, 0],
 ['saw', 19, 19],
 ['chara', 0, 1],
 ['ouat', 0, 1],
 ['sheet', 0, 1],
 ['msg', 2, 1],
 ['nisrina', 0, 1],
 ['katekyn', 0, 1],
 ['bucket', 0, 1],
 ['roller', 2, 0],
 ['avail', 10, 13],
 ['bigtim', 0, 1],
 ['d:', 0, 6],
 ['summertim', 0, 1],
 ['ale', 1, 0],
 ['omw', 0, 1],
 ['anythin', 0, 1],
 ['triathlet', 1, 0],
 ['tuesday', 7, 4],
 ['carrot', 0, 1],
 ['happi', 211, 25],
 ['nb', 1, 0],
 ['happybirthdayemilybett', 1, 0],
 ['south', 2, 2],
 ['awkward', 2, 1],
 ['denim', 1, 1],
 ['jax', 1, 0],
 ['belfast', 1, 1],
 ['noo', 0, 5],
 ['lion', 1, 3],
 ['ohaha', 1, 0],
 ['wahoo', 1, 0],
 ['cashback', 1, 0],
 ['influenti', 0, 1],
 ['immov', 0, 1],
 ['afso', 0, 1],
 ['announc', 1, 3],
 ['pakighinabi', 0, 1],
 ['sn', 1, 0],
 ['simpson', 0, 1],
 ['accid', 0, 2],
 ['carmel', 1, 0],
 ['bilal', 0, 1],
 ['jelli', 1, 0],
 ['mornin', 2, 0],
 ['neighbor', 0, 1],
 ['pokiri', 1, 0],
 ['idownload', 1, 0],
 ['actuallythough', 1, 0],
 [

# Feature Extraction

In [ ]:
def extract_features(tweet, freqs):
    
    # process_tweet tokenizes, stems, and removes stopwords
    word_l = preprocess_tweet(tweet)
    
    # 3 elements in the form of a 1 x 3 vector
    x = np.zeros((1, 3)) 
    
    #bias term is set to 1
    x[0,0] = 1 
       
    # loop through each word in the list of words
    for word in word_l:
         
        if (word,1.0) in freqs:
            # increment the word count for the positive label 1
            x[0,1] += freqs[(word,1.0)]
        if(word,0.0) in freqs:
            # increment the word count for the negative label 0
            x[0,2] += freqs[(word,0.0)]
        
    return x

In [36]:
# collect the features 'x' and stack them into a matrix 'X'
X = np.zeros((len(train_x), 3))
for i in range(len(train_x)):
    X[i, :]= extract_features(train_x[i], freqs)

# training labels corresponding to X
Y = train_y

print(X.shape)
print(Y.shape)

(8000, 3)
(8000, 1)


![alt text](image.png)

In [40]:
def sigmoid(z): 

    h = 1/(1+np.exp(-z))
   
    return h

In [41]:
def gradientDescent(x, y, theta, alpha, num_iters):
   
    m = x.shape[0]
    
    for i in range(0, num_iters):
        
        # get z, the dot product of x and theta
        z = np.dot(x,theta)
        
        # get the sigmoid of z
        h = sigmoid(z)
        
        # calculate the cost function
        J = -(np.dot(y.T,np.log(h))+np.dot((1-y).T,np.log(1-h)))/m

        # update the weights theta
        theta = theta - alpha*(np.dot(x.T,h-y))/m
        
    J = float(J)
    return J, theta

In [42]:
# Apply gradient descent
J, theta = gradientDescent(X, Y, np.zeros((3, 1)), 1e-9, 1500)

print(f"The cost after training is {J:.6f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(theta)]}")

The cost after training is 0.195138.
The resulting vector of weights is [7e-08, 0.00054223, -0.00054312]


/tmp/ipykernel_7595/1638565074.py:19: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  J = float(J)


# Testing the model

In [43]:
def predict_tweet(tweet, freqs, theta):

    # extract the features of the tweet and store it into x
    x = extract_features(tweet, freqs)
    
    # make the prediction using x and theta
    y_pred = sigmoid(np.dot(x,theta))
    
    return y_pred

In [44]:
# check your own sentiment
my_tweet = 'I love machine learning :)'
y_pred_temp = predict_tweet(my_tweet, freqs, theta)
print(y_pred_temp)

if y_pred_temp > 0.5:
    print('Positive sentiment')
else: 
    print('Negative sentiment')

[[0.8879634]]
Positive sentiment


In [45]:
def test_logistic_regression(test_x, test_y, freqs, theta):
    """ 
    test_x: a list of tweets
    test_y: (m, 1) vector with the corresponding labels for the list of tweets
    """
    
    y_hat = []
    
    for tweet in test_x:
        # get the label prediction for the tweet
        y_pred = predict_tweet(tweet, freqs, theta)
        
        if y_pred > 0.5:
            y_hat.append(1.0)
        else:
            y_hat.append(0.0)

    accuracy = np.sum(np.squeeze(test_y) == np.squeeze(np.asarray(y_hat)))/len(test_y)

    return accuracy

In [46]:
test_accuracy = test_logistic_regression(test_x, test_y, freqs, theta)
print(f"Logistic regression model's accuracy = {test_accuracy:.4f}")

Logistic regression model's accuracy = 0.9965


# Error Analysis

In [49]:
print('Label Predicted Tweet')
for x,y in zip(test_x,test_y):
    y_hat = predict_tweet(x, freqs, theta)

    if np.abs(y - (y_hat > 0.5)) > 0:
        # print('THE TWEET IS:', x)
        # print('THE PROCESSED TWEET IS:', process_tweet(x))
        print(y, y_hat)

Label Predicted Tweet
[1.] [[0.49884816]]
[1.] [[0.48216204]]
[1.] [[0.48216204]]
[1.] [[0.48216204]]
[1.] [[0.49669377]]
[1.] [[0.48163997]]
[0.] [[0.50103336]]
